In [1]:
from tweety.filters import SearchFilters
import nest_asyncio
from tweety import TwitterAsync
import toml
import os
from pathlib import Path
import pandas as pd

In [123]:
GLOBAL_CONFIG = {}

In [124]:
def load_config():
    with open("config/secrets.toml", "r") as toml_file:
        global GLOBAL_CONFIG
        GLOBAL_CONFIG = toml.load(toml_file)

In [125]:
load_config()

In [43]:
sess_file = Path(os.getcwd() + "/session.tw_session")

In [3]:
nest_asyncio.apply()

In [ ]:
app = TwitterAsync("session")

if not sess_file.is_file():
    print("Signing in to create session file.")

    x_twitter = GLOBAL_CONFIG["x"]
    await app.sign_in(x_twitter["username"], x_twitter["password"])

else:
    print("Authenticating using session file...")
    
await app.connect()

Signing in to create session file.


User(id=16975132, username=MohdFaizuddin, name=Faiz, verified=False)

In [28]:
search_keywords = "HMPV"

In [105]:
tweets = await app.search(search_keywords, pages=10, filter_=SearchFilters.Latest(), wait_time=5)

In [109]:
id = []
created_on = []
date = []
text = []
rich_text = []
author = []
is_retweet = []
retweeted_tweet = []
is_quoted = []
quoted_tweet = []
is_reply = []
is_sensitive = []
reply_counts = []
quote_counts = []
replied_to = []
bookmark_count = []
views = []
likes = []
language = []
place = []
retweet_counts = []
source = []
has_moderated_replies = []
is_liked = []
is_retweeted = []
can_reply = []
broadcast = []
edit_control = []
has_newer_version = []
audio_space_id = []
pool = []
community = []
media = []
user_mentions = []
urls = []
hashtags = []
symbols = []
community_note = []
url = []
grok_share = []
threads = []
comments = []


In [110]:
for t in tweets:
    id.append(t["id"])
    created_on.append(t["created_on"])
    date.append(t["date"])
    text.append(t["text"])
    rich_text.append(t["rich_text"])
    author.append(t["author"])
    is_retweet.append(t["is_retweet"])
    retweeted_tweet.append(t["retweeted_tweet"])
    is_quoted.append(t["is_quoted"])
    quoted_tweet.append(t["quoted_tweet"])
    is_reply.append(t["is_reply"])
    is_sensitive.append(t["is_sensitive"])
    reply_counts.append(t["reply_counts"])
    quote_counts.append(t["quote_counts"])
    replied_to.append(t["replied_to"])
    bookmark_count.append(t["bookmark_count"])
    views.append(t["views"])
    likes.append(t["likes"])
    language.append(t["language"])
    place.append(t["place"])
    retweet_counts.append(t["retweet_counts"])
    source.append(t["source"])
    has_moderated_replies.append(t["has_moderated_replies"])
    is_liked.append(t["is_liked"])
    is_retweeted.append(t["is_retweeted"])
    can_reply.append(t["can_reply"])
    broadcast.append(t["broadcast"])
    edit_control.append(t["edit_control"])
    has_newer_version.append(t["has_newer_version"])
    audio_space_id.append(t["audio_space_id"])
    pool.append(t["pool"])
    community.append(t["community"])
    media.append(t["media"])
    user_mentions.append(t["user_mentions"])
    urls.append(t["urls"])
    hashtags.append(t["hashtags"])
    symbols.append(t["symbols"])
    community_note.append(t["community_note"])
    url.append(t["url"])
    grok_share.append(t["grok_share"])
    threads.append(t["threads"])
    comments.append(t["comments"])

In [111]:
df = pd.DataFrame(list(zip(id,
                           created_on,
                           date,
                           text,
                           rich_text,
                           author,
                           is_retweet,
                           retweeted_tweet,
                           is_quoted,
                           quoted_tweet,
                           is_reply,
                           is_sensitive,
                           reply_counts,
                           quote_counts,
                           replied_to,
                           bookmark_count,
                           views,
                           likes,
                           language,
                           place,
                           retweet_counts,
                           source,
                           has_moderated_replies,
                           is_liked,
                           is_retweeted,
                           can_reply,
                           broadcast,
                           edit_control,
                           has_newer_version,
                           audio_space_id,
                           pool,
                           community,
                           media,
                           user_mentions,
                           urls,
                           hashtags,
                           symbols,
                           community_note,
                           url,
                           grok_share,
                           threads,
                           comments)), columns=["id",
                           "created_on",
                           "date",
                           "text",
                           "rich_text",
                           "author",
                           "is_retweet",
                           "retweeted_tweet",
                           "is_quoted",
                           "quoted_tweet",
                           "is_reply",
                           "is_sensitive",
                           "reply_counts",
                           "quote_counts",
                           "replied_to",
                           "bookmark_count",
                           "views",
                           "likes",
                           "language",
                           "place",
                           "retweet_counts",
                           "source",
                           "has_moderated_replies",
                           "is_liked",
                           "is_retweeted",
                           "can_reply",
                           "broadcast",
                           "edit_control",
                           "has_newer_version",
                           "audio_space_id",
                           "pool",
                           "community",
                           "media",
                           "user_mentions",
                           "urls",
                           "hashtags",
                           "symbols",
                           "community_note",
                           "url",
                           "grok_share",
                           "threads",
                           "comments"])

In [112]:
df

,id,created_on,date,text,rich_text,author,is_retweet,retweeted_tweet,is_quoted,quoted_tweet,...,media,user_mentions,urls,hashtags,symbols,community_note,url,grok_share,threads,comments
0,1876174893706047659,2025-01-06 07:52:06+00:00,2025-01-06 07:52:06+00:00,"HMPV virus in India--Two infants in Bengaluru,...",None,"{'id': '2333490811', 'rest_id': '2333490811', ...",False,None,False,None,...,[],[],[],[],[],None,https://twitter.com/shivam090195/status/187617...,None,[],[]
1,1876174873581564145,2025-01-06 07:52:02+00:00,2025-01-06 07:52:02+00:00,"""HMPV की दस्तक के बाद बाजार हिला, जानें Gold औ...",None,"{'id': '1390710458051485696', 'rest_id': '1390...",False,None,False,None,...,[],[],[{'display_url': 'thenewsair.com/market-shaken...,"[{'indices': [72, 77], 'text': 'news'}, {'indi...",[],None,https://twitter.com/TheNewsAir/status/18761748...,None,[],[]
2,1876174871807586539,2025-01-06 07:52:01+00:00,2025-01-06 07:52:01+00:00,#HMPV #vishwaguru #Teleprompter साहब पुरा देश ...,None,"{'id': '2862015775', 'rest_id': '2862015775', ...",False,None,False,None,...,[],[],[],"[{'indices': [0, 5], 'text': 'HMPV'}, {'indice...",[],None,https://twitter.com/nishu_sawant/status/187617...,None,[],[]
3,1876174871476236651,2025-01-06 07:52:01+00:00,2025-01-06 07:52:01+00:00,"Health Ministry Confirms, HMPV Cases Confirmed...",None,"{'id': '923168334920671232', 'rest_id': '92316...",False,None,False,None,...,[],[],[{'display_url': 'powercorridors.in/health-min...,"[{'indices': [62, 72], 'text': 'HMPVCases'}, {...",[],None,https://twitter.com/power_corridors/status/187...,None,[],[]
4,1876174864245239854,2025-01-06 07:51:59+00:00,2025-01-06 07:51:59+00:00,"हादसे अभी भी , आंखों को भूले नहीं हैं...!\nहम ...",None,"{'id': '1537378570330578949', 'rest_id': '1537...",False,None,False,None,...,"[{'display_url': 'pic.x.com/QUJeUwKzdI', 'expa...",[],[],"[{'indices': [108, 113], 'text': 'HMPV'}, {'in...",[],None,https://twitter.com/anuragv58203731/status/187...,None,[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,1876173708135670001,2025-01-06 07:47:24+00:00,2025-01-06 07:47:24+00:00,#HMPV is set to impact the tech industry! 🚀\n\...,None,"{'id': '1563064629370458112', 'rest_id': '1563...",False,None,False,None,...,[],[],[],"[{'indices': [0, 5], 'text': 'HMPV'}, {'indice...",[],None,https://twitter.com/karanjot_ji/status/1876173...,None,[],[]
179,1876173703504891957,2025-01-06 07:47:23+00:00,2025-01-06 07:47:23+00:00,भारत में ह्यूमन मेटान्यूमोवायरस (HMPV) के मामल...,"{'tags': [], 'id': 'Tm90ZVR3ZWV0OjE4NzYxNzM3MD...","{'id': '1840677759196057600', 'rest_id': '1840...",False,None,False,None,...,"[{'display_url': 'pic.x.com/sBV7dikjsN', 'expa...",[],[],"[{'indices': [477, 482], 'text': 'HMPV'}]",[],None,https://twitter.com/DrJyotsana51400/status/187...,None,[],[]
180,1876173698857849150,2025-01-06 07:47:22+00:00,2025-01-06 07:47:22+00:00,Le virus HMPV : Pas de danger détecté en Tunis...,None,"{'id': '394157532', 'rest_id': '394157532', 'c...",False,None,False,None,...,[],"[{'id': '16864182', 'name': 'L'Economiste', 's...",[{'display_url': 'leconomistemaghrebin.com/202...,[],[],None,https://twitter.com/economiste_magh/status/187...,None,[],[]
181,1876173692348375433,2025-01-06 07:47:20+00:00,2025-01-06 07:47:20+00:00,Human Metapneumovirus (HMPV): What You Need to...,None,"{'id': '1684417168723824640', 'rest_id': '1684...",False,None,False,None,...,"[{'display_url': 'pic.x.com/iuoJW5FYur', 'expa...",[],[],"[{'indices': [52, 59], 'text': 'Thread'}, {'in...",[],None,https://twitter.com/wahidansari1394/status/187...,None,[],[]


In [116]:
# append search keywords column
df["search_keywords"] = search_keywords

In [126]:
from pymongo import MongoClient

mongo_conf = GLOBAL_CONFIG["mongo"]
client = MongoClient(mongo_conf["mongo_uri"])
db = client[mongo_conf["mongo_db"]]
coll = mongo_conf["mongo_coll"]

In [127]:
def create_profiles_collection(db, collection_name, validator=None):
    if collection_name not in db.list_collection_names():
        if validator:
            db.create_collection(collection_name, validator=validator)
        else:
            db.create_collection(collection_name)
    else:
        print(f"Collection '{collection_name}' already exists.")

In [128]:
create_profiles_collection(db, coll)

In [129]:
def insert_data_to_mongo(db, collection_name, data_frame):
    collection = db[collection_name]
    records = data_frame.to_dict(orient='records')
    collection.insert_many(records)

In [130]:
insert_data_to_mongo(db, coll, df)

In [142]:
profiles_collection = db[coll]
cursor = profiles_collection.find()

In [143]:
list_cur = list(cursor)
dataset = pd.DataFrame(list_cur)

In [150]:
dataset.loc[1, "hashtags"]

[{'indices': [72, 77], 'text': 'news'},
 {'indices': [78, 91], 'text': 'BreakingNews'},
 {'indices': [92, 102], 'text': 'Headlines'},
 {'indices': [103, 117], 'text': 'CurrentEvents'},
 {'indices': [118, 128], 'text': 'WorldNews'},
 {'indices': [129, 138], 'text': 'Politics'},
 {'indices': [139, 152], 'text': 'BusinessNews'},
 {'indices': [153, 162], 'text': 'TechNews'},
 {'indices': [163, 174], 'text': 'HealthNews'},
 {'indices': [175, 186], 'text': 'SportsNews'},
 {'indices': [187, 205], 'text': 'EntertainmentNews'}]

### Text preprocessing

In [152]:
import malaya

/opt/anaconda3/envs/sentiment/lib/python3.11/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3397
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/opt/anaconda3/envs/sentiment/lib/python3.11/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3927
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


In [151]:
def preprocess_text(text):
    # demoji
    demoji = malaya.preprocessing.demoji()
    demoji_text = demoji.demoji(text)

    # malaya preprocessing
    preproc = malaya.preprocessing.preprocessing(demoji=demoji)
    preproc_text = "".join(preproc.process(text))

    for k,v in demoji_text.items():
        preproc_text = preproc_text.replace(k,v)

    w_tokeniser = malaya.tokenizer.Tokenizer()
    s_tokeniser = malaya.tokenizer.SentenceTokenizer()

    w_tokens = w_tokeniser.tokenize(preproc_text, lowercase=True)
    # s_tokens = s_tokeniser.tokenize(preproc_text)

    return "".join(w_tokens)

In [157]:
texts = []

for text in dataset["text"]:
    texts.append(preprocess_text(text))

In [160]:
import operator
fast_text = malaya.language_detection.fasttext()

langs = []
lang_probas = []

for text in texts:
    prob_dict = fast_text.predict_proba([text])

    lang = max(prob_dict[0].items(), key=operator.itemgetter(1))[0]
    prob = max(prob_dict[0].items(), key=operator.itemgetter(1))[1]

    langs.append(lang)
    lang_probas.append(prob)

ModuleNotFoundError: fasttext not installed. Please install it by `pip install fasttext` and try again.

## Sentiment analysis

In [161]:
malaya.sentiment.available_huggingface

{'mesolitica/sentiment-analysis-nanot5-tiny-malaysian-cased': {'Size (MB)': 93,
  'macro precision': 0.67768,
  'macro recall': 0.68266,
  'macro f1-score': 0.67997},
 'mesolitica/sentiment-analysis-nanot5-small-malaysian-cased': {'Size (MB)': 167,
  'macro precision': 0.67602,
  'macro recall': 0.6712,
  'macro f1-score': 0.67339}}

In [162]:
print(malaya.sentiment.info)

Trained on https://huggingface.co/datasets/mesolitica/chatgpt-explain-sentiment
Split 90% to train, 10% to test.


In [164]:
# load model from Huggingface
sentiment_model = malaya.sentiment.huggingface(model="mesolitica/sentiment-analysis-nanot5-small-malaysian-cased")

AttributeError: module 'inspect' has no attribute 'getargspec'